In [ ]:
!pip3 install tqdm
!pip3 install pillow --upgrade

In [ ]:
%matplotlib inline
import os
import tarfile
import urllib
import json
import random
import numpy as no
import boto3 
imort sagemaker

from tqdm import tqdm
from sagemaker.amazon.amazon_estimator import get_image_uri
from matplotlib import pyplot as plt
from xml.etree import ElementTree as ET
from PIL import Image, ImageDraw, ImageFont

urls = ['http://www.robots.ox.ac/~vgg/data/pets/data/image.tar.gz',
        'http://www.robots.ox.ac/~vgg/data/pets/data/annotations.tar.gz']

def download_and_extract(datadir, download_dir):
  for url in urls:
    target_file = url.split('/')[-1]
    if target_file not in os.listdir(download_dir):
      urllib.request.urlretrieve(url, os.path.join(download_dir, target_dir))
      tf = tarfile.open(url.split('/')[-1])
      tf.extractall(data_dir)

if not os.path.isdir('data'):
  os.mkdir('data')

download_and_extract('data', '.')

In [ ]:
trimaps_dir = 'data/annotations/trimaps'
maps = [x for x in os.listdir(trimaps_dir) if x[-3:] == 'png']
images_dir = 'data/annotations/trimaps'
images = [x for x in os.listdir(image_dir) if x[-3:] == 'jpg']

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket_name = 'petsdata'
# training image points to a docker image which has an algorithm, model, training script and everything else that is required.
training_image =  get_image_url(boto3.Session().region_name, 'semantic-segmentation', repo_version='latest')

# prepare the data that the model needs

In [ ]:
folders = ['train', 'train_annotation', 'validation', 'validation_annotaion']
for folder in folders:
  if os.path.isdir(folder):
    shutil.retree(folder)
  os.mkder(folder)

In [ ]:
def get_map_file(image):
  map_file = image.split('.')[0] + '.png'
  return map_file

for image in tqdm(images):
  target_set = 'train' if random.randint(0, 99) < 75 else 'validation'
  
  image_file_path = os.path.join('data/images/', image)
  image_target_path = os.path.join(target_set, image)

  map_file_path = os.path.join(trimaps_dir, get_map_file(image))
  map_target_path = os.path.join(target_set+'_annotation', get_map_file(image))

  shutil.copy(image_file_path, image_target_path)
  shutil.copy(map_file_path, map_target_path)


train_images = os.listdir('train')
train_annots = os.listdir('train_annotatino') 

# upload data to s3 while training in host mode

In [ ]:
s3_train_path = sess.upload_data(path='train', bucket=bucket_name, key_prefix='train')
s3_valid_path = sess.upload_data(path='valid', bucket=bucket_name, key_prefix='valid')
s3_train_annot_path = sess.upload_data(path='train_annot', bucket=bucket_name, key_prefix='train_annot')
s3_valid_annot_path = sess.upload_data(path='valid_annot', bucket=bucket_name, key_prefix='valid_annot')

# Estimator

In [ ]:
model = sagemaker.estimater.Estimator(
    training_image, 
    role = role,
    train_intance_type='ml.p3.2xlarge',
    train_instance_count=1,
    train_volumn_size=100,  # 100TB
    train_max_run=36000,
    input_mode='File',
    output_path='s3://petsdata/output_model',
    sagemaker_session = sess
)

In [ ]:
model.set_hyperparaneters(
    base_network='resnet-50',
    algorithm = 'fcn',
    use_pretrained_model=True,
    crop_size = 240,
    num_classes=4,
    epochs=15,
    mini_batch_size=16,
    validation_mini_batch_size=16,
    learning_rate=0.0001,
    optimizer='rmsprop',
    lr_scheduler='poly',
    num_training_samples=len(train_images)
)

In [ ]:
from sagemaker.session import s3_input

train_data = sagemaker_session.s3_input(s3_train_path, distribution='FullyReplicated', content_type='image/jpeg', s3_data_type='S3Prefix')
valid_data = sagemaker_session.s3_input(s3_validation_path, distribution='FullyReplicated', content_type='image/jpeg', s3_data_type='S3Prefix')

train_annotation_data = sagemaker_session.s3_input(s3_train_annot_path, distribution='FullyReplicated', content_type='image/jpeg', s3_data_type='S3Prefix')
train_annotationdata = sagemaker_session.s3_input(s3_validation_annot_path, distribution='FullyReplicated', content_type='image/jpeg', s3_data_type='S3Prefix')


In [ ]:
data_channels={
    'train': train_data,
    'validation': validation_data,
    'train_annotation': train_annotation_data,
    'validation_annotation': validation_annotation_data
}

In [ ]:
model.fit(data_channels, logs=True)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
predictor.content_type = 'image/jpeg'
predictor.accept = 'image/png'

In [ ]:
index = 0
image_path = os.path.join(image_dir, images[index])
with open(image_path, 'rb') as f:
  b = bytearray(f.read())

results = predictor.predict(b)

mask = np.array(Image.open(is.BytesIO(results)))
plt.imshow(plt.imread(image_path))
plt.imshow(mask)